In [1]:
import sys
import os

# Get the parent directory of the current folder
project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_dir not in sys.path:
    sys.path.insert(0, project_dir)


In [2]:
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize
from tqdm import tqdm
from scipy.stats import t, multivariate_normal, multivariate_t
from scipy.special import logsumexp, gamma, gammaln
from numpy.linalg import inv, det, slogdet, LinAlgError
import plotly.express as px
from plotly.subplots import make_subplots
from numpy import tril_indices, triu_indices
from typing import Literal, Optional, Dict, Union

# from simulation import * 

from Functions.simulation_1 import * 
from Functions.gas_filter_5 import *
from Functions.kf_filter_5 import *

## $\Phi$ Diagonal, $Q, R$ Full

In [3]:
T = 1000
N = 3
K = 3

c_true_3x3 = np.array([0.2, -0.1, 0.4])

Phi_true_3x3_phi_diag_yes_noise_correl = np.array([[0.8, 0.0, 0.0],
                         [0.0, 0.9, 0.0],
                         [0.0, 0.0, 0.7]])

beta_true_3x3 = np.array([[1.0, -0.5, 0.2],
                          [-1.2, 0.8, 0.4],
                          [0.5, 1.5, -0.3]])

Q_true_3x3_yes_correl = np.array([[0.7, 0.2, 0.3],
                                 [0.2, 0.5, 0.1],
                                 [0.3, 0.1, 0.6]])

R_true_3x3_yes_correl = np.array([[0.4, 0.1, 0.2],
                                 [0.1, 0.8, 0.3],
                                 [0.2, 0.3, 0.5]])

# Run explicit simulation with regressors
y_sim_3x3_phi_diag_yes_noise_correl, mu_sim_3x3_phi_diag_yes_noise_correl, X_sim_3x3_phi_diag_yes_noise_correl = simulate_multivariate_state_space(
    T=T,
    N=N,
    c=c_true_3x3,
    Phi=Phi_true_3x3_phi_diag_yes_noise_correl,
    beta=beta_true_3x3,  # Explicitly include regressors
    Q=Q_true_3x3_yes_correl,
    R=R_true_3x3_yes_correl,
    # use_intercept=None,
    seed=8888
)

print("✅ Multivariate State-Space Simulation with regressors completed.")

✅ Multivariate State-Space Simulation with regressors completed.


In [4]:
gas_results_3x3_phi_full_no_correl_1 = estimate_and_filter_gas(
    y=y_sim_3x3_phi_diag_yes_noise_correl, 
    X=X_sim_3x3_phi_diag_yes_noise_correl,
    phi_type= "diagonal",
    kappa_type= "full",
    fix_nu=None
    )


mu_gas_3x3_phi_full_no_correl_1 = gas_results_3x3_phi_full_no_correl_1["mu_filtered"]
u_gas_3x3_phi_full_no_correl_1 = gas_results_3x3_phi_full_no_correl_1["u"]
resid_gas_phi_full_no_correl_1 = gas_results_3x3_phi_full_no_correl_1["resid"]
Kappa_est_3x3_phi_full_no_correl_1 = gas_results_3x3_phi_full_no_correl_1["Kappa"]

Estimated omega:
 [ 0.0001279  -0.00862082 -0.10389872]
Estimated Phi:
 [[0.79794934 0.         0.        ]
 [0.         0.91242528 0.        ]
 [0.         0.         0.74881934]]
Estimated beta:
 [[ 1.97694164 -1.42913051  1.72979291]
 [-1.21080466  0.77110341  0.3742799 ]
 [ 0.54545373  1.56149953 -0.27223447]]
Estimated Omega (from lambda):
 [[1.27631457 0.         0.        ]
 [0.         1.40274576 0.        ]
 [0.         0.         1.24766092]]
Estimated Kappa:
 [[ 0.7634249   0.03113659  0.04785179]
 [ 0.07513064  0.67688293 -0.10055905]
 [-0.00583332 -0.08374802  0.62234112]]
Estimated nu:
 49.261752015733116
Log-likelihood: -4719.216035935517
AIC: 9494.432071871033
BIC: 9662.61036376524


In [5]:
# Dimensions
N = y_sim_3x3_phi_diag_yes_noise_correl.shape[1]
K = X_sim_3x3_phi_diag_yes_noise_correl.shape[1]

# Custom starting values
omega_init = np.array([0.1, 0.0, 0.2])                 # Shape (N,)
Phi_init = 0.5 * np.eye(N).flatten()                   # Shape (N*N,)
beta_init = 0.5 * np.ones((K, N)).flatten()                 # Shape (K*N,)

# Cholesky lower-triangle of Q and R (unconstrained)
L_Q_init = np.ones(N * (N + 1) // 2)                   # Length N*(N+1)/2
L_R_init = np.ones(N * (N + 1) // 2)

# Concatenate into full initial parameter vector
initial_params = np.concatenate([omega_init, Phi_init, beta_init, L_Q_init, L_R_init])

kf_results_3x3_phi_full_no_correl  =  multivariate_KF_with_estimation(
    y=y_sim_3x3_phi_diag_yes_noise_correl,
    X=X_sim_3x3_phi_diag_yes_noise_correl,
    initial_params=initial_params,
    verbose=True
)

mu_kf_3x3_phi_full_no_correl = kf_results_3x3_phi_full_no_correl['mu_filtered']
P_kf_3x3_phi_full_no_correl = kf_results_3x3_phi_full_no_correl['P_filtered']
kalman_gain_kf_3x3_phi_full_no_correl = kf_results_3x3_phi_full_no_correl['kalman_gain']

✅ Kalman estimation completed successfully.
Estimated parameters:
omega (unconditional mean): [ 0.43018053 -0.48581514  0.44593007]
Phi (persistence matrix): 
[[ 7.91481524e-01  1.28203016e-02 -3.11092585e-03]
 [-6.40297679e-04  9.07486645e-01 -8.38217104e-03]
 [-6.04022649e-02  3.60937379e-02  7.63462569e-01]]
beta: 
[[ 1.55732803 -0.93570741  1.18004541]
 [-1.21056671  0.77225922  0.37015156]
 [ 0.5521405   1.5643806  -0.26784229]]
Q (state noise covariance): 
[[0.78326961 0.19184788 0.37855916]
 [0.19184788 0.41262395 0.09332609]
 [0.37855916 0.09332609 0.5900256 ]]
R (observation noise covariance): 
[[0.380411   0.13161197 0.16927432]
 [0.13161197 0.756644   0.28919032]
 [0.16927432 0.28919032 0.53968973]]
Log-likelihood: -4538.345586227106
AIC: 9142.691172454211
BIC: 9340.90130218667


In [6]:
gas_specs = [
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": None, "label": "GAS_diag_diag_nufree"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 5, "label": "GAS_diag_diag_nu5"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 10, "label": "GAS_diag_diag_nu10"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 20, "label": "GAS_diag_diag_nu20"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 50, "label": "GAS_diag_diag_nu50"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 100, "label": "GAS_diag_diag_nu100"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 500, "label": "GAS_diag_diag_nu500"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": None, "label": "GAS_diag_full_nufree"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 5, "label": "GAS_diag_full_nu5"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 10, "label": "GAS_diag_full_nu10"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 20, "label": "GAS_diag_full_nu20"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 50, "label": "GAS_diag_full_nu50"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 100, "label": "GAS_diag_full_nu100"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 500, "label": "GAS_diag_full_nu500"},
    # Add more specs as needed
]

In [7]:
# from concurrent.futures import ProcessPoolExecutor, as_completed
# import numpy as np

# def run_single_simulation(sim_seed, T, N, c, Phi, beta, Q, R, gas_specs):
#     np.random.seed(sim_seed)
#     results = {"seed": sim_seed, "KF": None, "GAS": {}}
#     # Simulate data
#     y_sim, mu_sim, X_sim = simulate_multivariate_state_space(
#         T=T,
#         N=N,
#         c=c,
#         Phi=Phi,
#         beta=beta,
#         Q=Q,
#         R=R,
#         seed=sim_seed,
#     )
#     # KF
#     try:
#         kf_results = multivariate_KF_with_estimation(
#             y=y_sim, 
#             X=X_sim, 
#             initial_params=None, 
#             verbose=False,
#             ftol=1e-8,
#             gtol=1e-5,
#             optim_method='2stage'
#         )
#         mu_kf = kf_results['mu_filtered']
#         mse_kf = np.mean((mu_sim[1:] - mu_kf[1:]) ** 2)
#         mae_kf = np.mean(np.abs(mu_sim[1:] - mu_kf[1:]))
#         results["KF"] = {
#             "loglik": kf_results["loglik"],
#             "aic": kf_results["aic"],
#             "bic": kf_results["bic"],
#             "mse": mse_kf,
#             "mae": mae_kf,
#         }
#     except Exception as e:
#         results["KF"] = {"error": str(e)}
#     # GAS
#     for spec in gas_specs:
#         label = spec["label"]
#         try:
#             res_gas = estimate_and_filter_gas(
#                 y=y_sim, 
#                 X=X_sim,
#                 phi_type=spec["phi_type"], 
#                 kappa_type=spec["kappa_type"],
#                 fix_nu=spec["fix_nu"], 
#                 verbose=False,
#                 ftol=1e-8,
#                 gtol=1e-5,
#             )
#             mu_gas = res_gas["mu_filtered"]
#             mse_gas = np.mean((mu_sim[1:] - mu_gas[1:]) ** 2)
#             mae_gas = np.mean(np.abs(mu_sim[1:] - mu_gas[1:]))
#             results["GAS"][label] = {
#                 "loglik": res_gas["loglik"],
#                 "aic": res_gas["aic"],
#                 "bic": res_gas["bic"],
#                 "mse": mse_gas,
#                 "mae": mae_gas,
#             }
#         except Exception as e:
#             results["GAS"][label] = {"error": str(e)}
#     return results


In [8]:
# from concurrent.futures import ProcessPoolExecutor, as_completed, ThreadPoolExecutor
# from collections import defaultdict

# T = 500
# target_success = 10
# spec_results = defaultdict(list)
# pending_labels = ["KF"] + [spec["label"] for spec in gas_specs]
# sim_counter = 0
# max_parallel = 8  # Or set to os.cpu_count() for max cores

# # with ProcessPoolExecutor(max_workers=max_parallel) as executor:
# with ThreadPoolExecutor(max_workers=8) as executor:
#     submitted = 0
#     futures = []
#     # Submit initial batch
#     while len(futures) < max_parallel and any(len(spec_results[l]) < target_success for l in pending_labels):
#         sim_seed = 8888 + sim_counter
#         sim_counter += 1
#         futures.append(executor.submit(
#             run_single_simulation,
#             sim_seed, T, N, c_true_3x3, Phi_true_3x3_phi_diag_yes_noise_correl,
#             beta_true_3x3, Q_true_3x3_yes_correl, R_true_3x3_yes_correl, gas_specs
#         ))
#     # As futures complete, submit new ones
#     while any(len(spec_results[l]) < target_success for l in pending_labels):
#         for future in as_completed(futures):
#             result = future.result()
#             # Aggregate as before
#             if "error" not in result["KF"] and len(spec_results["KF"]) < target_success:
#                 spec_results["KF"].append({**result["KF"], "seed": result["seed"]})
#             for label, gas_res in result["GAS"].items():
#                 if "error" not in gas_res and len(spec_results[label]) < target_success:
#                     spec_results[label].append({**gas_res, "seed": result["seed"]})
#             # Check if more are needed
#             if all(len(spec_results[l]) >= target_success for l in pending_labels):
#                 break
#             # Submit a new one for each finished
#             sim_seed = 8888 + sim_counter
#             sim_counter += 1
#             new_future = executor.submit(
#                 run_single_simulation,
#                 sim_seed, T, N, c_true_3x3, Phi_true_3x3_phi_diag_yes_noise_correl,
#             beta_true_3x3, Q_true_3x3_yes_correl, R_true_3x3_yes_correl, gas_specs
#             )
#             futures.append(new_future)
#         # Remove completed futures from the list
#         futures = [f for f in futures if not f.done()]
#         # If done, break outer loop
#         if all(len(spec_results[l]) >= target_success for l in pending_labels):
#             break


In [9]:
from collections import defaultdict

spec_results = defaultdict(list)
target_success = 30
sim_counter = 0

while any(len(v) < target_success for v in spec_results.values()) or len(spec_results) == 0:
    sim_seed = 8888 + sim_counter
    sim_counter += 1
    # Simulate data
    # Run explicit simulation with regressors
    y_sim, mu_sim, X_sim = simulate_multivariate_state_space(
            T=T,
            N=N,
            c=c_true_3x3,
            Phi=Phi_true_3x3_phi_diag_yes_noise_correl,
            beta=beta_true_3x3,  # Explicitly include regressors
            Q=Q_true_3x3_yes_correl,
            R=R_true_3x3_yes_correl,
            # use_intercept=None,
            seed=sim_seed,
        )


    # KF
    try:
        if len(spec_results["KF"]) < target_success:
            kf_results = multivariate_KF_with_estimation(
            y=y_sim, 
            X=X_sim, 
            initial_params=None, 
            verbose=False,
            ftol=1e-8,  # Tolerance for convergence
            gtol=1e-6,  # Gradient tolerance for convergence,
        )
            # Calculate MSE for mu
            mu_kf = kf_results['mu_filtered']
            mse_kf = np.mean((mu_sim[1:] - mu_kf[1:]) ** 2)
            mae_kf = np.mean(np.abs(mu_sim[1:] - mu_kf[1:]))

            spec_results["KF"].append({
                "seed": sim_seed, "loglik": kf_results["loglik"],
                "aic": kf_results["aic"], "bic": kf_results["bic"],
                "mse": mse_kf, "mae": mae_kf
                })

    except Exception as e:
        print(f"KF failed: {e}")
    # GAS
    for spec in gas_specs:
        try:
            label = spec["label"]
            if len(spec_results[label]) < target_success:
                res_gas = estimate_and_filter_gas(
                    y=y_sim, 
                    X=X_sim,
                    phi_type=spec["phi_type"], 
                    kappa_type=spec["kappa_type"],
                    fix_nu=spec["fix_nu"], 
                    verbose=False,
                    ftol=1e-8,  # Tolerance for convergence
                    gtol=1e-6   # Gradient tolerance for convergence
                )

                # Calculate MSE for mu
                mu_gas = res_gas["mu_filtered"]
                mse_gas = np.mean((mu_sim[1:] - mu_gas[1:]) ** 2)
                mae_gas = np.mean(np.abs(mu_sim[1:] - mu_gas[1:]))

                spec_results[label].append({
                    "seed": sim_seed, "loglik": res_gas["loglik"],
                    "aic": res_gas["aic"], "bic": res_gas["bic"],
                    "mse": mse_gas, "mae": mae_gas
                })
        except Exception as e:
            print(f"GAS {label} failed: {e}")

    # Break if all filled
    if all(len(v) >= target_success for v in spec_results.values()):
        break

# Now you can flatten/aggregate spec_results as needed


In [10]:
import pandas as pd

# Flatten to long-form list of dicts
rows = []
for spec_label, run_list in spec_results.items():
    for run in run_list:
        row = {'spec': spec_label}
        row.update(run)  # adds 'seed', 'loglik', etc.
        rows.append(row)

df_results = pd.DataFrame(rows)


agg = df_results.groupby('spec').agg({
    'loglik': ['mean', 'std'],
    'aic': ['mean', 'std'],
    'bic': ['mean', 'std'],
    'mse': ['mean', 'std'],  # Mean Squared Error
    'mae': ['mean', 'std'],  # Mean Absolute Error
    # add more metrics here if you wish!
}).reset_index()
# Flatten MultiIndex columns
agg.columns = ['spec', 'loglik_mean', 'loglik_std', 'aic_mean', 'aic_std', 'bic_mean', 'bic_std', 'mse_mean', 'mse_std', 'mae_mean', 'mae_std']  # Added mse and mae


agg

,spec,loglik_mean,loglik_std,aic_mean,aic_std,bic_mean,bic_std,mse_mean,mse_std,mae_mean,mae_std
0,GAS_diag_diag_nu10,-4726.888668,44.504968,9495.777336,89.009936,9621.911055,89.009936,1.114022,0.104025,0.886462,0.051027
1,GAS_diag_diag_nu100,-4706.953505,45.014154,9455.907010,90.028308,9582.040729,90.028308,0.693713,0.048527,0.674326,0.026849
2,GAS_diag_diag_nu20,-4709.052855,44.463986,9460.105710,88.927971,9586.239429,88.927971,0.898559,0.077785,0.782432,0.040831
3,GAS_diag_diag_nu5,-4770.426276,45.200538,9594.852552,90.401077,9757.024476,90.401077,1.369289,0.129228,0.998272,0.058827
4,GAS_diag_diag_nu50,-4705.772751,44.799487,9453.545502,89.598974,9579.679221,89.598974,0.744876,0.056175,0.702221,0.030728
5,GAS_diag_diag_nu500,-4709.144419,45.232908,9460.288838,90.465816,9586.422556,90.465816,0.650453,0.042775,0.650326,0.023881
6,GAS_diag_diag_nufree,-4705.147089,44.678169,9454.294179,89.356338,9586.434265,89.356338,2.860247,1.789362,1.486893,0.498985
7,GAS_diag_full_nu10,-4717.470513,44.838295,9488.941026,89.676590,9651.112950,89.676590,1.105345,0.104235,0.883941,0.051435
8,GAS_diag_full_nu100,-4697.176534,45.730474,9448.353069,91.460948,9610.524993,91.460948,0.685711,0.048339,0.671090,0.026935
9,GAS_diag_full_nu20,-4699.717673,45.014575,9453.435345,90.029150,9615.607270,90.029150,0.890571,0.077171,0.779821,0.040609
